[Reference](https://medium.com/@datajournal/web-scraping-with-chatgpt-aab68342b849)

In [2]:
import requests
from bs4 import BeautifulSoup

def get_refurbished_iphone_urls():
    url = "https://www.apple.com/shop/refurbished/iphone"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Failed to retrieve the page")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    product_links = []

    for link in soup.find_all("a", href=True):
        if "/shop/product/" in link["href"]:
            full_url = "https://www.apple.com" + link["href"]
            product_links.append(full_url)
    return product_links

In [3]:
import json
import traceback
from bs4 import BeautifulSoup
import requests

def scrape_iphone_details(url):
    """Scrapes iPhone data from a given Apple product page."""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
    # Extract product name, price, and stock status from schema.org JSON-LD
        json_ld_script = soup.find("script", type="application/ld+json")
        product_data = json.loads(json_ld_script.string)
        phone_name = product_data.get("name", "Unknown Model")
        price = product_data.get("offers", {}).get("price", "Unknown Price")
        stock_status = "In Stock" if "InStock" in product_data.get("offers", {}).get("availability", "") else "Out of Stock"
        return {
            "Phone Model Name": phone_name,
            "Price": price,
            "Stock Status": stock_status
        }
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

In [4]:
import time
from datetime import datetime
import os

def price_monitor():
    """Monitors price changes daily."""
    while True:
        print("Running daily price check…")

        # Get list of URLs
        urls = get_refurbished_iphone_urls()

        # Scrape data from each URL
        iphone_data = []

        for url in urls:
            details = scrape_iphone_details(url)
        if details:
            iphone_data.append(details)
        # Save the data to a CSV or markdown file
        timestamp = datetime.now().strftime("%Y-%m-%d")
        file_path = f"iphone_prices_{timestamp}.csv"
        with open(file_path, 'w') as file:
            for data in iphone_data:
                file.write(f"{data['Phone Model Name']},{data['Price']},{data['Stock Status']}\n")
            print(f"Data saved to {file_path}")

        # Wait for 24 hours before running again
        time.sleep(86400)

if __name__ == "__main__":
    price_monitor()

Running daily price check…
Error scraping https://www.apple.com/shop/product/FQ3R3LL/A/refurbished-iphone-14-plus-128gb-midnight-unlocked?fnode=89d3d652d4a6944753f4dd838586541cca2c2ea93367e860281611303e14607240e664a7b88bdae5a9f6b9c26c9c46a62e602e982962c2d8941caf1b24909172e0d5f8cfaef718ada77b94766c6ae2a30219e8b145750fda5b0e4d1ac1b5dda4: 'list' object has no attribute 'get'
Error scraping https://www.apple.com/shop/product/FQ1K3LL/A/Refurbished-iPhone-14-Pro-512GB-Space-Black-Unlocked?fnode=89d3d652d4a6944753f4dd838586541cca2c2ea93367e860281611303e14607240e664a7b88bdae5a9f6b9c26c9c46a62e602e982962c2d8941caf1b24909172e0d5f8cfaef718ada77b94766c6ae2a30219e8b145750fda5b0e4d1ac1b5dda4: 'list' object has no attribute 'get'
Error scraping https://www.apple.com/shop/product/FQ213LL/A/Refurbished-iPhone-14-Pro-512GB-Gold-Unlocked?fnode=89d3d652d4a6944753f4dd838586541cca2c2ea93367e860281611303e14607240e664a7b88bdae5a9f6b9c26c9c46a62e602e982962c2d8941caf1b24909172e0d5f8cfaef718ada77b94766c6ae2a3021

KeyboardInterrupt: 